In [2]:
!pwd

/workspaces/cosc470/notebooks


In [ ]:
import torch
from torchvision import datasets, transforms

mnist_train = datasets.MNIST(
    root='/workspaces/cosc470/data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

mnist_test = datasets.MNIST(
    root='/workspaces/cosc470/data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)



100%|██████████| 9.91M/9.91M [00:00<00:00, 108MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 7.19MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.7MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.90MB/s]


In [15]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=4)
        self.pool = nn.MaxPool2d(5, 5)
        self.fc1 = nn.Linear(20 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 20 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

model = SimpleCNN()
bad_prediction = model.forward(torch.randn(1, 1, 28, 28))
print(bad_prediction)
real_image = mnist_train[0][0].unsqueeze(0)
print(mnist_train[0][1])
good_prediction = model.forward(real_image)
print(good_prediction)

tensor([[0.1639, 0.0871, 0.0733, 0.0880, 0.0706, 0.1155, 0.0938, 0.1313, 0.1035,
         0.0729]], grad_fn=<SoftmaxBackward0>)
5
tensor([[0.1187, 0.1030, 0.0879, 0.1020, 0.0976, 0.1047, 0.0954, 0.1018, 0.0971,
         0.0917]], grad_fn=<SoftmaxBackward0>)


In [16]:
from torch.utils.data import DataLoader

# Set hyperparameters
learning_rate = 0.01
batch_size = 10
epochs = 20

# DataLoader for training
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    correct = 0
    for testimage in mnist_test:
        a = testimage[0].unsqueeze(0)
        a = model.forward(a)
        if torch.argmax(a) == testimage[1]:
            correct += 1
    print(f"Test Accuracy: {100 * correct / len(mnist_test):.2f}%")

Epoch [1/20], Loss: 2.0103
Test Accuracy: 81.26%
Epoch [2/20], Loss: 1.6016
Test Accuracy: 92.09%
Epoch [3/20], Loss: 1.5458
Test Accuracy: 94.25%
Epoch [4/20], Loss: 1.5253
Test Accuracy: 95.31%
Epoch [5/20], Loss: 1.5136
Test Accuracy: 95.80%
Epoch [6/20], Loss: 1.5065
Test Accuracy: 96.40%
Epoch [7/20], Loss: 1.5017
Test Accuracy: 97.09%
Epoch [8/20], Loss: 1.4980
Test Accuracy: 97.19%
Epoch [9/20], Loss: 1.4950
Test Accuracy: 97.15%
Epoch [10/20], Loss: 1.4929
Test Accuracy: 97.54%
Epoch [11/20], Loss: 1.4911
Test Accuracy: 97.51%
Epoch [12/20], Loss: 1.4894
Test Accuracy: 97.58%
Epoch [13/20], Loss: 1.4880
Test Accuracy: 97.53%
Epoch [14/20], Loss: 1.4865
Test Accuracy: 97.83%
Epoch [15/20], Loss: 1.4856
Test Accuracy: 97.73%
Epoch [16/20], Loss: 1.4844
Test Accuracy: 97.76%
Epoch [17/20], Loss: 1.4834
Test Accuracy: 97.99%
Epoch [18/20], Loss: 1.4825
Test Accuracy: 98.13%
Epoch [19/20], Loss: 1.4818
Test Accuracy: 97.90%
Epoch [20/20], Loss: 1.4811
Test Accuracy: 98.23%
